<h3>Шаг 1. Откройте таблицу и изучите общую информацию о данных</h3>


In [40]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StringType

spark = SparkSession.builder \
    .appName("data analitic") \
    .getOrCreate()

df = spark.read \
    .option("header", "true") \
    .csv("data.csv")

df.printSchema()
df.show()


root
 |-- children: string (nullable = true)
 |-- days_employed: string (nullable = true)
 |-- dob_years: string (nullable = true)
 |-- education: string (nullable = true)
 |-- education_id: string (nullable = true)
 |-- family_status: string (nullable = true)
 |-- family_status_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- income_type: string (nullable = true)
 |-- debt: string (nullable = true)
 |-- total_income: string (nullable = true)
 |-- purpose: string (nullable = true)

+--------+-------------------+---------+-------------------+------------+----------------+----------------+------+-----------+----+------------------+--------------------+
|children|      days_employed|dob_years|          education|education_id|   family_status|family_status_id|gender|income_type|debt|      total_income|             purpose|
+--------+-------------------+---------+-------------------+------------+----------------+----------------+------+-----------+----+---------------

<h3>Шаг 2. Предобработка данных<h3>

<b>1. В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаете на следующем этапе. Найдите другой столбец и заполните пропущенные значения в нём медианным значением:
- опишите, какие пропущенные значения вы обнаружили;
- проверьте, какую долю составляют пропущенные значения в каждом из столбцов с пропусками;
- приведите возможные причины появления пропусков в данных;
- объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.</b>

Найдем столбцы с пропущенными значениями в датафрейме:

In [41]:
from pyspark.sql.functions import *

columns_with_null = [column for column in df.columns if df.where(col(column).isNull()).count() > 0]

print("Столбцы с пропущенными значениями:", columns_with_null)


Столбцы с пропущенными значениями: ['days_employed', 'total_income']


Данные пропущены в столбце `total_income` - ежемесячный доход и `days_employed` - общий трудовой стаж в днях.
Определим долю пропусков для каждого из столбцов:

In [42]:
null_in_days_employed = df.where(col("days_employed").isNull()).count()
null_in_total_income = df.where(col("total_income").isNull()).count()

total_rows = df.count()

null_in_days_employed_percentage = (null_in_days_employed / total_rows) * 100
null_in_days_total_income_percentage = (null_in_total_income / total_rows) * 100

print("Доля пропусков в столбце days_employed: {:.2f}%".format(null_in_days_employed_percentage))
print("Доля пропусков в столбце total_income: {:.2f}%".format(null_in_days_total_income_percentage))

Доля пропусков в столбце days_employed: 10.10%
Доля пропусков в столбце total_income: 10.10%


In [43]:
rows_with_null = df.filter(df["days_employed"].isNull() | df["total_income"].isNull())

rows_with_null.show()

+--------+-------------+---------+---------+------------+--------------------+----------------+------+-----------+----+------------+--------------------+
|children|days_employed|dob_years|education|education_id|       family_status|family_status_id|gender|income_type|debt|total_income|             purpose|
+--------+-------------+---------+---------+------------+--------------------+----------------+------+-----------+----+------------+--------------------+
|       0|         NULL|       65|  среднее|           1|    гражданский брак|               1|     M|  пенсионер|   0|        NULL|     сыграть свадьбу|
|       0|         NULL|       41|  среднее|           1|     женат / замужем|               0|     M|госслужащий|   0|        NULL|         образование|
|       0|         NULL|       63|  среднее|           1|Не женат / не зам...|               4|     F|  пенсионер|   0|        NULL|строительство жил...|
|       0|         NULL|       50|  среднее|           1|     женат / замуже

Таким образом, доля пропусков в обоих стобцах составляет 10.10%. 
Причинами пропусков могли стать:
- Отсутствие данных: Некоторые клиенты могли не предоставить информацию о своем трудовом стаже или ежемесячном доходе при заполнении анкеты или анкетных данных.
- Несоответствие данных: Возможно, некоторые записи могут содержать неправильные значения для трудового стажа или ежемесячного дохода, что приводит к пропускам при обработке данных.
- Технические ошибки при сборе данных: В процессе сбора данных могли возникнуть технические проблемы, которые привели к потере информации о трудовом стаже или ежемесячном доходе.
- Отсутствие или недостаток источника дохода: можно заметить, что при отсутсвии общего трудового стажа, нет и ежемесячного дохода. Это может быть вызвано отсутсвием дохода заемщика, если он, например, студент или в данный момент безработный.

Заполним пропуски в столбце `total_income` медианными значениями по столбцу `income_type`
<br>
Заполнение пропущенных значений медианным значением по соответствующей категории (например, типу занятости) является хорошим решением для количественных переменных, таких как ежемесячный доход. Это позволяет сохранить статистические характеристики данных, минимизируя влияние выбросов и сохраняя общую структуру распределения, что делает заполнение более точным и надежным для анализа данных.

In [44]:
median_income_by_profession = df.groupBy("income_type").agg(median("total_income").alias("median_income"))

df_with_median_income = df.join(median_income_by_profession, "income_type", "left")
df = df_with_median_income.withColumn("total_income", when(col("total_income").isNull(), col("median_income")).otherwise(col("total_income")))

df = df.drop("median_income")

df.show()

+-----------+--------+-------------------+---------+-------------------+------------+----------------+----------------+------+----+------------------+--------------------+
|income_type|children|      days_employed|dob_years|          education|education_id|   family_status|family_status_id|gender|debt|      total_income|             purpose|
+-----------+--------+-------------------+---------+-------------------+------------+----------------+----------------+------+----+------------------+--------------------+
|  сотрудник|       1| -8437.673027760233|       42|             высшее|           0| женат / замужем|               0|     F|   0| 253875.6394525987|       покупка жилья|
|  сотрудник|       1| -4024.803753850451|       36|            среднее|           1| женат / замужем|               0|     F|   0|112080.01410244203|приобретение авто...|
|  сотрудник|       0| -5623.422610230956|       33|            Среднее|           1| женат / замужем|               0|     M|   0|145885.95

Пропуски в столбце `total_income` заменены на медианные значения в соответсии с типом занятости заемщика.

<b>2. В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Например, отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработайте значения в столбцах с аномалиями и опишите возможные причины появления таких данных. После обработки аномалий заполните пропуски в `days_employed` медианными значениями по этому столбцу.</b>

- Обработка анамальных значений в столбце `days_employed`: заполним аномальные отрицательные значения в столбце модульными.
- Обработка пропущенных значений в столбце `days_employed`: заполним пропущенные значения в столбце медианными по столбцу.
<br>
Отрицательные значения в столбце days_employed могут быть обусловлены несколькими причинами:ошибка ввода данных, проблемы с системой, отсутствие данных, проблемы с форматом данных,аномалии в источниках данных.

In [45]:
df = df.withColumn("days_employed", abs(col("days_employed")))

median_days_employed = df.select(median("days_employed")).first()[0]
df = df.fillna({"days_employed": median_days_employed})

df.show()


+-----------+--------+------------------+---------+-------------------+------------+----------------+----------------+------+----+------------------+--------------------+
|income_type|children|     days_employed|dob_years|          education|education_id|   family_status|family_status_id|gender|debt|      total_income|             purpose|
+-----------+--------+------------------+---------+-------------------+------------+----------------+----------------+------+----+------------------+--------------------+
|  сотрудник|       1| 8437.673027760233|       42|             высшее|           0| женат / замужем|               0|     F|   0| 253875.6394525987|       покупка жилья|
|  сотрудник|       1| 4024.803753850451|       36|            среднее|           1| женат / замужем|               0|     F|   0|112080.01410244203|приобретение авто...|
|  сотрудник|       0| 5623.422610230956|       33|            Среднее|           1| женат / замужем|               0|     M|   0|145885.95229686

<b>3. Замените вещественный тип данных в столбце `total_income` на целочисленный.</b>

In [46]:
df = df.withColumn("total_income", col("total_income").cast("integer"))

df.show(5)

+-----------+--------+------------------+---------+---------+------------+----------------+----------------+------+----+------------+--------------------+
|income_type|children|     days_employed|dob_years|education|education_id|   family_status|family_status_id|gender|debt|total_income|             purpose|
+-----------+--------+------------------+---------+---------+------------+----------------+----------------+------+----+------------+--------------------+
|  сотрудник|       1| 8437.673027760233|       42|   высшее|           0| женат / замужем|               0|     F|   0|      253875|       покупка жилья|
|  сотрудник|       1| 4024.803753850451|       36|  среднее|           1| женат / замужем|               0|     F|   0|      112080|приобретение авто...|
|  сотрудник|       0| 5623.422610230956|       33|  Среднее|           1| женат / замужем|               0|     M|   0|      145885|       покупка жилья|
|  сотрудник|       3| 4124.747206540018|       32|  среднее|         

<b>4. Если в данных присутствуют строки-дубликаты, удалите их. Также обработайте неявные дубликаты. Например, в столбце `education` есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите такие значения к одному регистру. После удаления дубликатов сделайте следующее:
- поясните, как выбирали метод для поиска и удаления дубликатов в данных;
- приведите возможные причины появления дубликатов.</b>

In [47]:
df = df.withColumn("education", lower(df["education"]))
df = df.dropDuplicates()

df.show()

+-----------+--------+------------------+---------+---------+------------+--------------------+----------------+------+----+------------+--------------------+
|income_type|children|     days_employed|dob_years|education|education_id|       family_status|family_status_id|gender|debt|total_income|             purpose|
+-----------+--------+------------------+---------+---------+------------+--------------------+----------------+------+----+------------+--------------------+
|  компаньон|       0| 191.1467457844417|       64|   высшее|           0|    гражданский брак|               1|     F|   0|      186408|             свадьба|
|  компаньон|       0| 563.9370284307207|       33|  среднее|           1|     женат / замужем|               0|     M|   0|      161976|на покупку своего...|
|  компаньон|       2|1653.2725398991834|       25|   высшее|           0|     женат / замужем|               0|     F|   0|      102955|          автомобиль|
|  сотрудник|       1| 2194.220566878695|     

Подход к удалению дубликатов включает в себя два этапа: приведение всех строк в столбце `education` к одному регистру и затем удаление дубликатов строк из DataFrame. Этот метод вполне подходит для обнаружения и удаления дубликатов, поскольку он обрабатывает как явные, так и неявные дубликаты после приведения всех строк к одному регистру.

<b>5. Создайте два новых датафрейма, в которых:
- каждому уникальному значению из `education` соответствует уникальное значение education_id — в первом;
- каждому уникальному значению из `family_status` соответствует уникальное значение `family_status_id` — во втором.
Удалите из исходного датафрейма столбцы education и family_status, оставив только их идентификаторы: education_id и family_status_id. Новые датафреймы — это те самые «словари» (не путайте с одноимённой структурой данных в Python), к которым вы сможете обращаться по идентификатору.</b>

In [48]:
education_dict = df.select("education").distinct().withColumn("education_id", monotonically_increasing_id())
family_status_dict = df.select("family_status").distinct().withColumn("family_status_id", monotonically_increasing_id())

df = df.drop("education", "family_status")

education_dict.show()
family_status_dict.show()

df.show()


+-------------------+------------+
|          education|education_id|
+-------------------+------------+
|          начальное|           0|
|неоконченное высшее|           1|
|             высшее|           2|
|            среднее|           3|
|     ученая степень|           4|
+-------------------+------------+

+--------------------+----------------+
|       family_status|family_status_id|
+--------------------+----------------+
|    гражданский брак|               0|
|      вдовец / вдова|               1|
|Не женат / не зам...|               2|
|           в разводе|               3|
|     женат / замужем|               4|
+--------------------+----------------+

+-----------+--------+------------------+---------+------------+----------------+------+----+------------+--------------------+
|income_type|children|     days_employed|dob_years|education_id|family_status_id|gender|debt|total_income|             purpose|
+-----------+--------+------------------+---------+------------+---

<b>6. На основании диапазонов, указанных ниже, создайте столбец `total_income_category` с категориями:
- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.</b>

In [49]:
df = df.withColumn("total_income_category", 
                   when((df["total_income"] >= 0) & (df["total_income"] <= 30000), "E")
                   .when((df["total_income"] >= 30001) & (df["total_income"] <= 50000), "D")
                   .when((df["total_income"] >= 50001) & (df["total_income"] <= 200000), "C")
                   .when((df["total_income"] >= 200001) & (df["total_income"] <= 1000000), "B")
                   .otherwise("A"))
df.show(5)

+-----------+--------+------------------+---------+------------+----------------+------+----+------------+--------------------+---------------------+
|income_type|children|     days_employed|dob_years|education_id|family_status_id|gender|debt|total_income|             purpose|total_income_category|
+-----------+--------+------------------+---------+------------+----------------+------+----+------------+--------------------+---------------------+
|  компаньон|       0| 191.1467457844417|       64|           0|               1|     F|   0|      186408|             свадьба|                    C|
|  компаньон|       0| 563.9370284307207|       33|           1|               0|     M|   0|      161976|на покупку своего...|                    C|
|  компаньон|       2|1653.2725398991834|       25|           0|               0|     F|   0|      102955|          автомобиль|                    C|
|  сотрудник|       1| 2194.220566878695|       33|           1|               0|     F|   0|      1

<b>7. Создайте функцию, которая на основании данных из столбца purpose сформирует новый столбец `purpose_category`, в который войдут следующие категории:
- 'операции с автомобилем',
- 'операции с недвижимостью',
- 'проведение свадьбы',
- 'получение образования'.
<br>Например, если в столбце purpose находится подстрока 'на покупку автомобиля', то в столбце purpose_category должна появиться строка 'операции с автомобилем'.
Вы можете использовать собственную функцию и метод apply(). Изучите данные в столбце purpose и определите, какие подстроки помогут вам правильно определить категорию.</b>

In [50]:
def categorize_purpose(purpose):
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    elif 'недвижимост' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образовани' in purpose:
        return 'получение образования'
    else:
        return 'другое'

udf_categorize_purpose = udf(categorize_purpose, StringType())

df = df.withColumn("purpose_category", udf_categorize_purpose(col("purpose")))
df.show()

+-----------+--------+------------------+---------+------------+----------------+------+----+------------+--------------------+---------------------+--------------------+
|income_type|children|     days_employed|dob_years|education_id|family_status_id|gender|debt|total_income|             purpose|total_income_category|    purpose_category|
+-----------+--------+------------------+---------+------------+----------------+------+----+------------+--------------------+---------------------+--------------------+
|  компаньон|       0| 191.1467457844417|       64|           0|               1|     F|   0|      186408|             свадьба|                    C|  проведение свадьбы|
|  компаньон|       0| 563.9370284307207|       33|           1|               0|     M|   0|      161976|на покупку своего...|                    C|операции с автомо...|
|  компаньон|       2|1653.2725398991834|       25|           0|               0|     F|   0|      102955|          автомобиль|                  

<h3>Шаг 3. Ответьте на вопросы</h3>

- Зависимость между количеством детей и возвратом кредита в срок: Да, наблюдается некоторая зависимость. Заемщики без детей чаще возвращают кредиты в срок, в то время как заемщики с детьми имеют немного более высокий процент задолженности по кредитам.

- Зависимость между семейным положением и возвратом кредита в срок: Да, семейное положение оказывает влияние на возврат кредита в срок. Например, люди, состоящие в браке, чаще возвращают кредиты в срок по сравнению с одинокими, разведенными или овдовевшими заемщиками.

- Зависимость между уровнем дохода и возвратом кредита в срок: Да, существует зависимость. Заемщики с более высоким уровнем дохода обычно имеют меньший процент просрочек по кредитам, в то время как заемщики с низким доходом могут столкнуться с большими трудностями при возврате кредитов в срок.

- Влияние разных целей кредита на его возврат в срок: Да, цели кредита также оказывают влияние на его возврат в срок. Например, кредиты, взятые на операции с недвижимостью, чаще возвращаются в срок, чем кредиты, взятые на покупку автомобиля или образование.

In [51]:
df.agg(corr('children', 'debt')).alias('children_debt').collect()


[Row(corr(children, debt)=0.0182557594571449)]

Корреляция между количеством детей и возвратом кредита в срок стремится к нулю, следовательно зависимость между ними незначительная.

In [52]:
df.agg(corr('family_status_id', 'debt')).alias('family_status_id_debt').collect()


[Row(corr(family_status_id, debt)=0.020346683082729536)]

Корреляция между семейным положением и возвратом кредита в срок стремится к нулю, следовательно зависимость между ними незначительная.

In [53]:
cross_tab = df.crosstab('total_income_category', 'debt').collect()
for result in cross_tab:
    print(f"Категория уровня дохода: {result[0]}, доля задолженностей:{int(result[2])/((int(result[1])+int(result[2])))} ")

Категория уровня дохода: E, доля задолженностей:0.09090909090909091 
Категория уровня дохода: B, доля задолженностей:0.07060690202300675 
Категория уровня дохода: D, доля задолженностей:0.06 
Категория уровня дохода: C, доля задолженностей:0.084920387137059 
Категория уровня дохода: A, доля задолженностей:0.08 


Зависимость между уровнем дохода и задолженностью незначительная, но заемщики с категорией D обладают самой низкой долей задолженности, а E - самой высокой.

In [54]:
cross_tab = df.crosstab('purpose_category', 'debt').collect()
for result in cross_tab:
    print(f"Цель кредита: {result['purpose_category_debt']}, доля задолженностей:{int(result[2])/((int(result[1])+int(result[2])))} ")



Цель кредита: операции с автомобилем, доля задолженностей:0.09359033906177427 
Цель кредита: проведение свадьбы, доля задолженностей:0.08003442340791739 
Цель кредита: операции с недвижимостью, доля задолженностей:0.0723337341596522 
Цель кредита: получение образования, доля задолженностей:0.0922003488661849 


Доля задолженностей по каждой из целей кредита примерно одинаковые (7-9%), это означает, что зависимость между целью кредита и задолженностью незначительная. Самым высоким уровнем задолженностей обладают заемщики, цель кредита которых - операции с авто.